In [61]:
!pwd

/Users/ashish1610dhiman/data_projects/fall22_hw/RoboChef/notebooks/recommendation


In [147]:
from sklearn.manifold import TSNE

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
import pandas as pd
import numpy as np
from surprise import accuracy, Dataset, SVD,SVDpp, NMF
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split as train_test_split_surprise
from tqdm.notebook import tqdm

import numpy as np
from sklearn.metrics import ndcg_score

In [112]:
import sys
from os.path import dirname
sys.path.append(dirname("../../"))

from src.utilities import *
from src.reccomendation_util import *

In [113]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load DataSet

In [65]:
recipe_metadata = pd.read_csv("../../data/recommendation/RAW_recipes.csv")
recipe_metadata.shape

(231637, 13)

In [66]:
raw_interactions = pd.read_csv("../../data/recommendation/RAW_interactions.csv")
raw_interactions.shape

(1132367, 5)

In [67]:
test_set = pd.read_csv("../../data/recommendation/ad_interaction_test.csv")
test_set.shape

(244884, 5)

In [68]:
raw_subset= pd.read_csv("../../data/reccomendation_subset_data.csv")
raw_subset.shape

(872021, 5)

### Subset Test Data

In [70]:
test_subset = test_set[test_set["user_id"].isin(set(raw_subset["user_id"]))]
test_subset.shape

(174103, 5)

### Load Models

In [73]:
model1 = load_pickle("../../models/reccomender_model1_svd.pkl")
model2 = load_pickle("../../models/reccomender_model2_nmf.pkl")

### get predictions for a user

In [80]:
reader = Reader(rating_scale=(1, 5))

In [83]:
test_set_surprise_subset = Dataset.load_from_df(test_subset[["user_id", "recipe_id", "rating"]], reader)
_, test_set_surprise_subset = train_test_split_surprise(test_set_surprise_subset, test_size=1.0)

In [84]:
predictions1_sub = model1.test(test_set_surprise_subset)
predictions2_sub = model2.test(test_set_surprise_subset)

### Create Prediction Ranking

In [133]:
prediction_df1 = pd.DataFrame(predictions1_sub)
prediction_df1.shape

(174103, 5)

In [141]:
set(test_subset["user_id"])-set(prediction_df1["uid"])

set()

In [139]:
prediction_df1

,uid,iid,r_ui,est,details
0,52282,13303,4.0,4.524859,{'was_impossible': False}
1,283251,117490,3.0,4.134002,{'was_impossible': False}
2,121185,157032,4.0,4.213591,{'was_impossible': False}
3,363393,66241,5.0,4.590136,{'was_impossible': False}
4,133174,55406,5.0,4.291369,{'was_impossible': False}
...,...,...,...,...,...
174098,194829,38325,5.0,4.640050,{'was_impossible': False}
174099,95743,47454,5.0,4.195931,{'was_impossible': False}
174100,626447,107760,5.0,4.246153,{'was_impossible': False}
174101,1537420,108364,5.0,4.606777,{'was_impossible': False}


In [248]:
prediction_df2 = pd.DataFrame(predictions2_sub)
prediction_df2.shape

(174103, 5)

In [249]:
df_rank1 = get_prediction_ranking(prediction_df1)
df_rank1.shape

(20049, 3)

In [250]:
df_rank2 = get_prediction_ranking(prediction_df2)
df_rank2.shape

(20049, 3)

In [251]:
df_rank2.head()

,uid,iid,est
0,1533,"[57283, 54632, 30979, 10554, 86007, 41707, 755...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ..."
1,1535,"[162765, 122289, 378543, 198942, 80975, 99752,...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ..."
2,1634,"[82625, 361216, 44432, 26212, 9115, 28816, 127...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ..."
3,1676,"[7913, 99156, 825, 36561, 204082, 179836, 3423...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.415..."
4,1792,"[87440, 32844, 111459, 91105, 87544, 405535, 8...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ..."


### Create True Ranking 

In [172]:
pd.DataFrame.from_dict({user:[get_true_ranking_user(test_subset, raw_subset, recipe_metadata, 1533)]},\
                      orient = "index")

,0
2059,"[59135, 41707, 57549, 57283, 23891, 17854, 833..."


In [175]:
rank_i

,0
2148,"[59135, 41707, 57549, 57283, 23891, 17854, 833..."


In [183]:
true_rank_df = pd.DataFrame()
for user in tqdm(df_rank2["uid"].unique()):
    rank_i = pd.DataFrame.from_dict({user:[get_true_ranking_user(test_subset,\
                                                                 raw_subset, recipe_metadata, user)]\
                                    },orient = "index")
    true_rank_df = pd.concat([true_rank_df,rank_i])

  0%|          | 0/20049 [00:00<?, ?it/s]

In [186]:
true_rank_df = true_rank_df.reset_index()
true_rank_df.columns = ["user_id","ranked_recipe_list"]

In [187]:
true_rank_df.head()

,user_id,ranked_recipe_list
0,1533,"[59135, 41707, 57549, 57283, 23891, 17854, 833..."
1,1535,"[834, 112614, 119660, 33501, 74394, 26370, 713..."
2,1634,"[9115, 44432, 26212, 16193, 8709, 361216, 2881..."
3,1676,"[204082, 4022, 34233, 136683, 126128, 36561, 1..."
4,1792,"[32844, 405535, 88836, 91105, 31020, 111459, 8..."


In [189]:
true_rank_df.to_csv("../../data/true_ranking_test_data.csv",index=False)

### Ranking for reccomendation

In [154]:
a = get_true_ranking_user(test_subset, raw_subset, recipe_metadata, 1533)

In [155]:
b = df_rank2["iid"][0]

In [156]:
set(a)-set(b)

set()

In [158]:
rank_order= [b.index(true_item)+1 for true_item in a]
print(rank_order)

[7, 6, 1, 10, 9, 22, 20, 5, 14, 25, 8, 11, 26, 18, 16, 19, 3, 21, 24, 27, 15, 4, 2, 23, 12, 13, 28, 17]


In [164]:
ndcg_score(np.asarray([list(range(1,len(a)+1))]),y_score = np.asarray([rank_order]))

0.9074661552623133

In [165]:
ndcg_score(np.asarray([list(range(1,len(a)+1))]),y_score = np.asarray([rank_order[::-1]]))

0.7440817272876236

### evaluation of Reccomendation on Test Set
1. NDCG,NDCG @ k
2. MAP @ k

In [252]:
def get_ndcg(a,b,k=-1):
    true_rank = list(range(1,len(a)+1))
    pred_rank = [b.index(true_item)+1 for true_item in a]
    if len(a)==1 and len(set(a)-set(b))==0:
        return 1
    if k>0:
        return ndcg_score(np.asarray([true_rank]),y_score = np.asarray([pred_rank]), k = k)
    else:
        return ndcg_score(np.asarray([true_rank]),y_score = np.asarray([pred_rank]))

In [253]:
get_ndcg(a,b), get_ndcg(a,b,5)

(0.9074661552623133, 0.7398919830029868)

In [254]:
df_rank2.shape,true_rank_df.shape

((20049, 3), (20049, 2))

In [255]:
rank_merged1 = df_rank1.merge(true_rank_df,left_on="uid",right_on="user_id")
rank_merged2 = df_rank2.merge(true_rank_df,left_on="uid",right_on="user_id")
rank_merged1.shape,rank_merged2.shape

((20049, 5), (20049, 5))

In [256]:
rank_merged1.head()

,uid,iid,est,user_id,ranked_recipe_list
0,1533,"[57549, 44459, 31048, 10554, 8857, 41707, 5913...","[4.984623436267778, 4.979205256134577, 4.90942...",1533,"[59135, 41707, 57549, 57283, 23891, 17854, 833..."
1,1535,"[79278, 307072, 155496, 384168, 5483, 69914, 2...","[4.887660730957626, 4.869646919374931, 4.85833...",1535,"[834, 112614, 119660, 33501, 74394, 26370, 713..."
2,1634,"[132011, 11641, 28851, 44432, 9115, 12713, 120...","[3.830791080322248, 3.8093206066643965, 3.8043...",1634,"[9115, 44432, 26212, 16193, 8709, 361216, 2881..."
3,1676,"[126128, 825, 34233, 7913, 99156, 165679, 4022...","[4.914768923312829, 4.8161532228860695, 4.7314...",1676,"[204082, 4022, 34233, 136683, 126128, 36561, 1..."
4,1792,"[87440, 88836, 88497, 91105, 111459, 32844, 89...","[5.0, 4.796666814631808, 4.604151827864249, 4....",1792,"[32844, 405535, 88836, 91105, 31020, 111459, 8..."


In [257]:
rank_merged1["ndcg_overall"] = rank_merged1.apply(lambda row:\
                                                  get_ndcg(row.ranked_recipe_list,row.iid, k),axis=1)

rank_merged2["ndcg_overall"] = rank_merged2.apply(lambda row:\
                                                  get_ndcg(row.ranked_recipe_list,row.iid),axis=1)

In [258]:
rank_merged1[["ndcg_overall"]].describe()

,ndcg_overall
count,20049.000000
mean,0.915253
std,0.083943
min,0.687158
25%,0.859719
50%,0.922495
75%,1.000000
max,1.000000


In [259]:
rank_merged2[["ndcg_overall"]].describe()

,ndcg_overall
count,20049.000000
mean,0.915288
std,0.083897
min,0.671807
25%,0.859719
50%,0.922495
75%,1.000000
max,1.000000
